In [2]:
# for colab
# !pip install --upgrade datasets
# # !pip install --upgrade ipywidgets

from huggingface_hub import login
hf_token = ''
login(hf_token)

C:\Users\jc\.conda\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\jc\.cache\huggingface\token
Login successful


In [3]:
import random
import tensorflow as tf
import numpy as np
from datasets import load_dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional

In [4]:
# SOLO PARA USO EN GOOGLE COLABORATORY
# para conectar el notebook con la cuenta de gdrive
# from google.colab import drive
# drive.mount('/content/drive/')
# !ls "/content/drive/My Drive/"
# CHECKPOINTS_FOLDER = '/content/drive/MyDrive/TFM/checkpoints' # Se debe garantizar que la carpeta docencia compartida se almacena en el directorio raíz de Google Drive. En caso contrario modificar este path
# MODELS_FOLDER = '/content/drive/MyDrive/TFM/saved_models' # Se debe garantizar que la carpeta docencia compartida se almacena en el directorio raíz de Google Drive. En caso contrario modificar este path

# for localhost
CHECKPOINTS_FOLDER = './checkpoints'
MODELS_FOLDER = './saved_models'

In [5]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        print("Found a GPU with the name:", gpu)
else:
    print("Failed to detect a GPU.")


Found a GPU with the name: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [5]:
# specific language
ds = load_dataset("bigcode/the-stack-dedup", data_dir="data/python", streaming=True, split="train", revision="main")

README.md:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/144 [00:00<?, ?it/s]

In [8]:
VERSION = 'v4'
DATASETS = 80

# load datasets
counter = 0
corpus = ""

for sample in iter(ds):
    content = sample['content']
    if content.isascii(): # ignore files with asian symbols
      corpus += content
      counter+=1
      if counter == DATASETS:
          break

splitted = corpus.split('\n')
to_delete = []
flag = False
for index, line in enumerate(splitted):
    stripped = line.strip()
    if '"""' in line:
        flag = not flag

    if flag or \
        '>>>' in line or \
        ':param' in line or \
        stripped.startswith('#'):
      to_delete.append(index)

print("Removed lines:", len(to_delete))

for index in sorted(to_delete, reverse=True):
    del splitted[index]

corpus = '\n'.join(splitted)

Removed lines: 605


In [9]:
# Vectorize text

maxlen = 60 # sequences length
step = 3 # letters by sequence

sentences = []
next_chars = []

for i in range(0, len(corpus) - maxlen, step):
    sentences.append(corpus[i: i + maxlen])
    next_chars.append(corpus[i + maxlen])

# Letters corpus creation
unique_chars = sorted(list(set(corpus)))
char_indices = {char : i for i,char in enumerate(unique_chars) }
print(char_indices)
print(len(unique_chars))
len(corpus)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '5': 21, '7': 22, '8': 23, '9': 24, ':': 25, '<': 26, '=': 27, '>': 28, '@': 29, 'A': 30, 'B': 31, 'C': 32, 'D': 33, 'E': 34, 'F': 35, 'G': 36, 'H': 37, 'I': 38, 'J': 39, 'K': 40, 'L': 41, 'M': 42, 'N': 43, 'O': 44, 'P': 45, 'R': 46, 'S': 47, 'T': 48, 'U': 49, 'V': 50, 'W': 51, 'Y': 52, 'Z': 53, '[': 54, '\\': 55, ']': 56, '^': 57, '_': 58, '`': 59, 'a': 60, 'b': 61, 'c': 62, 'd': 63, 'e': 64, 'f': 65, 'g': 66, 'h': 67, 'i': 68, 'j': 69, 'k': 70, 'l': 71, 'm': 72, 'n': 73, 'o': 74, 'p': 75, 'q': 76, 'r': 77, 's': 78, 't': 79, 'u': 80, 'v': 81, 'w': 82, 'x': 83, 'y': 84, 'z': 85, '{': 86, '}': 87}
88


52431

In [ ]:
# Vectorization (one hot encoding)
x = np.zeros((len(sentences), maxlen, len(unique_chars)), dtype=bool) # every sequence, hot encoded
y = np.zeros((len(sentences), len(unique_chars)), dtype=bool) # for every sequence, hot encoded next character
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print(x.shape)
print(y.shape)

(67780, 60, 97)
(67780, 97)


In [ ]:
# LSTM model
model = models.Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(unique_chars)), return_sequences=True)) # returns 128 dim vector sequence
# # model.add(LSTM(64)) # v1
# # model.add(Bidirectional(LSTM(64))) # v2
# model.add(LSTM(64, recurrent_dropout=0.25)) # v3, v4
model.add(LSTM(64, recurrent_dropout=0.25, return_sequences=True)) # v5
model.add(LSTM(64, recurrent_dropout=0.25)) # v5
model.add(layers.Dense(len(unique_chars), activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 60, 256)             │         362,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 60, 64)              │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 97)                  │           6,305 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 484,001 (1.85 MB)

 Trainable params: 484,001 (1.85 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train model
# history = model.load_weights(f"{CHECKPOINTS_FOLDER}/test_{VERSION}_datasets_{DATASETS}.weights.h5")
history = model.fit(x, y, batch_size=128, epochs=1000, callbacks=[
    tf.keras.callbacks.ModelCheckpoint(
        filepath=f"{CHECKPOINTS_FOLDER}/test_{VERSION}_datasets_{DATASETS}.weights.h5",
        save_weights_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="acc",
        min_delta=0.01,
        patience=3
    )
])

# model = tf.keras.models.load_model(f'{MODELS_FOLDER}/test_{VERSION}_datasets_{DATASETS}.keras')
model.save(f'{MODELS_FOLDER}/test_{VERSION}_datasets_{DATASETS}.keras')

Epoch 1/1000
530/530 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - acc: 0.2431 - loss: 3.3286
Epoch 1: saving model to /content/drive/MyDrive/TFM/checkpoints_hito2/test_v4_datasets_80.weights.h5
530/530 ━━━━━━━━━━━━━━━━━━━━ 111s 204ms/step - acc: 0.2431 - loss: 3.3283
Epoch 2/1000
530/530 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - acc: 0.2668 - loss: 2.9315
Epoch 2: saving model to /content/drive/MyDrive/TFM/checkpoints_hito2/test_v4_datasets_80.weights.h5
530/530 ━━━━━━━━━━━━━━━━━━━━ 140s 201ms/step - acc: 0.2668 - loss: 2.9314
Epoch 3/1000
530/530 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - acc: 0.3085 - loss: 2.6976
Epoch 3: saving model to /content/drive/MyDrive/TFM/checkpoints_hito2/test_v4_datasets_80.weights.h5
530/530 ━━━━━━━━━━━━━━━━━━━━ 146s 208ms/step - acc: 0.3085 - loss: 2.6975
Epoch 4/1000
530/530 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - acc: 0.3458 - loss: 2.5327
Epoch 4: saving model to /content/drive/MyDrive/TFM/checkpoints_hito2/test_v4_datasets_80.weights.h5
530/530 ━━━━━━━━━━━━━━━━━━━━ 142s 

In [ ]:
# from google.colab import runtime
# runtime.unassign()

In [ ]:
def transform_distribution(predictions, temperature=0.5):
    """
      Calculate character position from prediction
    """
    predictions = np.array(predictions).astype('float64')
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions) # check if values sum 1 (probability)
    probs = np.random.multinomial(1, predictions, 1)
    return np.argmax(probs)

In [ ]:
def predict_code(input_text, predict_length=500, temperature=0.35):
    print('Seed: ' + input_text)
    generated_text = input_text

    for i in range(predict_length): # generate as many characters as requested
        sampled = np.zeros((1, maxlen, len(unique_chars))) # create array for one hot encoded input text
        for t, char in enumerate(input_text): # for every char in input_text
            sampled[0, t, char_indices[char]] = 1. # one hot encode input text

        prediction = model.predict(sampled, verbose=0)[0] # predict next character for one hot encoded input text
        next_index = transform_distribution(prediction, temperature) # get character most likely to be the next
        next_char = unique_chars[next_index] # get character from vectorized chars vector

        input_text += next_char # add char to input text
        generated_text += next_char # add char to final variable
        input_text = input_text[1:] # remove first char from input text and continue

    return generated_text

In [ ]:
# generar secuencias arbitrarias de texto
predict_length = 500

# temperaturas altas: mas entropia (mas aleatorio)
# temperaturas bajas: menos estocasticidad (mas deterministico)
temperature = 0.2

# Test 1
"""
    - Param: def get_label(
    - Expected:
    def get_label(code="", label=""):
        if code is None:
            code = "-"
        if label is None:
            label = "inconnu"
        return f"{code} {label[:30]}"
"""
start_index = 334
input_text = corpus[start_index : start_index + maxlen]
# print(input_text)
prediction = predict_code(input_text, predict_length, temperature)
print(prediction)

Seed: def get_label(code="", label=""):
    if code is None:
     
def get_label(code="", label=""):
    if code is None:
        self._vim.command('dtaync', *selacted.)
            self._vim.command('dyin.not relf._winheight
                                                                                                                                                                                                                                                                                                                                                                                                                 
